# PCB Defect Detection: Systematic Study Results Analysis

## Comprehensive Analysis of YOLOv8 Variants, Attention Mechanisms, and Loss Functions

This notebook provides automated analysis of experimental results from Weights & Biases, focusing on:
- **Performance Comparison**: mAP@0.5, mAP@0.5-0.95, precision, recall, F1 score
- **Efficiency Analysis**: Training time, inference speed, model parameters
- **Trade-off Visualization**: Pareto plots for edge deployment optimization
- **Model Selection**: Optimal configurations for different deployment scenarios

---

**Experiment Overview:**
- **Phase 1**: Baseline model comparison (YOLOv8n, YOLOv8s, YOLOv10s)
- **Phase 2**: Attention mechanisms (ECA, CBAM, CoordAtt)
- **Phase 3**: Loss functions and high-resolution studies

**Dataset**: HRIPCB (6 PCB defect classes)
**Hardware**: 22GB GPU, 50GB RAM

## 1. Setup and Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import os
import getpass
from PIL import Image
import io
import base64

# WandB API with authentication
try:
    import wandb
    print("✅ WandB imported successfully")
    WANDB_AVAILABLE = True
except ImportError:
    print("❌ WandB not found. Install with: !pip install wandb")
    WANDB_AVAILABLE = False

# Statistical analysis
try:
    from scipy import stats
    from scipy.spatial.distance import pdist, squareform
    print("✅ SciPy imported successfully")
except ImportError:
    print("❌ SciPy not found. Install with: !pip install scipy")

# Advanced plotting
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.offline as pyo
    pyo.init_notebook_mode(connected=True)
    print("✅ Plotly imported successfully")
    PLOTLY_AVAILABLE = True
except ImportError:
    print("⚠️  Plotly not available. Install with: !pip install plotly")
    PLOTLY_AVAILABLE = False

# Configure matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10

# Configure seaborn
sns.set_palette("husl")

print("\n🚀 Setup complete! Ready for analysis.")

## 2. Configuration

In [ ]:
# WandB Configuration
WANDB_PROJECT = "pcb-defect-150epochs-v1"
WANDB_ENTITY = None  # Set to your username if needed

# Authentication settings
WANDB_API_KEY = None  # Will be set during authentication
WANDB_AUTHENTICATED = False

# Analysis Configuration
SAVE_PLOTS = True
OUTPUT_DIR = Path("analysis_results")
OUTPUT_DIR.mkdir(exist_ok=True)

# Edge Deployment Constraints
EDGE_CONSTRAINTS = {
    'max_model_size_mb': 50,      # Maximum model size for edge deployment
    'min_fps': 10,                # Minimum inference speed requirement
    'min_map50': 0.60,           # Minimum acceptable mAP@0.5
    'max_inference_time_ms': 100  # Maximum inference time per image
}

# Color scheme for consistency
MODEL_COLORS = {
    'yolov8n': '#1f77b4',
    'yolov8s': '#ff7f0e', 
    'yolov10s': '#2ca02c'
}

ATTENTION_COLORS = {
    'none': '#1f77b4',
    'eca': '#ff7f0e',
    'cbam': '#2ca02c',
    'coordatt': '#d62728'
}

print(f"📊 Project: {WANDB_PROJECT}")
print(f"📁 Output directory: {OUTPUT_DIR}")
print(f"⚡ Edge constraints: {EDGE_CONSTRAINTS}")

# Display authentication status
print(f"🔐 WandB Available: {WANDB_AVAILABLE}")
if WANDB_AVAILABLE:
    print(f"🔐 Authentication Status: {'✅ Ready' if WANDB_AUTHENTICATED else '❌ Required'}")

## 3. Data Fetching Functions

In [ ]:
def fetch_wandb_runs(project_name, entity=None):
    """
    Fetch all runs from WandB project and extract comprehensive metrics with enhanced validation.
    
    Returns:
        pd.DataFrame: Comprehensive experiment results
    """
    if not WANDB_AUTHENTICATED:
        print("❌ WandB not authenticated. Please run authentication first.")
        return pd.DataFrame()
    
    print("🔄 Fetching runs from WandB with comprehensive data validation...")
    
    try:
        # Initialize WandB API
        api = wandb.Api()
        
        # Get project path
        project_path = f"{entity}/{project_name}" if entity else project_name
        runs = api.runs(project_path)
        
        print(f"📥 Found {len(runs)} total runs")
        
        # Filter for finished runs only
        finished_runs = [run for run in runs if run.state == 'finished']
        print(f"✅ Found {len(finished_runs)} finished runs for analysis")
        
        if len(finished_runs) == 0:
            print("❌ No finished runs found. Cannot proceed with analysis.")
            return pd.DataFrame()
        
        # Extract run data with enhanced error handling
        runs_data = []
        failed_runs = []
        
        for i, run in enumerate(finished_runs):
            if i % 5 == 0:  # Progress indicator
                print(f"   Processing run {i+1}/{len(finished_runs)}...")
                
            try:
                # Basic run information
                run_data = {
                    'run_id': run.id,
                    'run_name': run.name,
                    'state': run.state,
                    'created_at': run.created_at,
                    'duration_seconds': run._attrs.get('runtime', 0),
                    'tags': run.tags,
                    'notes': run.notes,
                    'url': run.url
                }
                
                # Extract configuration with validation
                config = run.config if hasattr(run, 'config') and run.config else {}
                
                # Model configuration
                model_config = config.get('model_config', {})
                run_data.update({
                    'model_type': model_config.get('type', 'unknown'),
                    'attention_mechanism': model_config.get('attention_mechanism', 'none'),
                    'model_config_path': model_config.get('config_path', ''),
                    'pretrained': model_config.get('pretrained', True)
                })
                
                # Training configuration  
                training_config = config.get('training_config', {})
                run_data.update({
                    'batch_size': training_config.get('batch', 'unknown'),
                    'image_size': training_config.get('imgsz', 640),
                    'epochs': training_config.get('epochs', 'unknown'),
                    'optimizer': training_config.get('optimizer', 'unknown'),
                    'learning_rate': training_config.get('lr0', 'unknown'),
                    'weight_decay': training_config.get('weight_decay', 'unknown'),
                    'amp': training_config.get('amp', True),
                    'cache': training_config.get('cache', True)
                })
                
                # Loss configuration
                loss_config = training_config.get('loss', {})
                run_data.update({
                    'loss_type': loss_config.get('type', 'standard'),
                    'box_weight': loss_config.get('box_weight', 7.5),
                    'cls_weight': loss_config.get('cls_weight', 0.5),
                    'dfl_weight': loss_config.get('dfl_weight', 1.5)
                })
                
                # Data configuration
                data_config = config.get('data_config', {})
                run_data['num_classes'] = data_config.get('num_classes', 6)
                
                # Extract final metrics from summary with validation
                summary = run.summary if hasattr(run, 'summary') and run.summary else {}
                
                # Performance metrics with multiple possible keys
                metrics_mapping = {
                    'map50': ['metrics/mAP50', 'metrics/mAP50(B)', 'val/mAP50', 'mAP50'],
                    'map50_95': ['metrics/mAP50-95', 'metrics/mAP50-95(B)', 'val/mAP50-95', 'mAP50-95'],
                    'precision': ['metrics/precision(B)', 'metrics/precision', 'val/precision', 'precision'],
                    'recall': ['metrics/recall(B)', 'metrics/recall', 'val/recall', 'recall'],
                    'f1_score': ['metrics/F1(B)', 'metrics/F1', 'val/F1', 'f1'],
                    'box_loss': ['train/box_loss', 'box_loss'],
                    'cls_loss': ['train/cls_loss', 'cls_loss'], 
                    'dfl_loss': ['train/dfl_loss', 'dfl_loss'],
                    'val_box_loss': ['val/box_loss'],
                    'val_cls_loss': ['val/cls_loss'],
                    'val_dfl_loss': ['val/dfl_loss'],
                    'total_parameters': ['model/total_parameters'],
                    'trainable_parameters': ['model/trainable_parameters'],
                    'training_time_seconds': ['final/training_time_seconds', 'training_time']
                }
                
                # Extract metrics with fallback and validation
                metrics_found = 0
                for metric_name, possible_keys in metrics_mapping.items():
                    value = None
                    for key in possible_keys:
                        if key in summary and summary[key] is not None:
                            value = summary[key]
                            metrics_found += 1
                            break
                    run_data[metric_name] = value
                
                # Calculate GFLOPs
                if run_data.get('model_type') and run_data.get('image_size'):
                    gflops = calculate_gflops(run_data['model_type'], run_data['image_size'])
                    run_data['gflops'] = gflops
                else:
                    run_data['gflops'] = None
                    
                # Speed metrics (if available)
                if 'speed/inference' in summary:
                    run_data['inference_time_ms'] = summary['speed/inference']
                    run_data['fps'] = 1000.0 / summary['speed/inference'] if summary['speed/inference'] > 0 else None
                elif 'speed' in summary and isinstance(summary['speed'], dict):
                    speed_dict = summary['speed']
                    if 'inference' in speed_dict:
                        run_data['inference_time_ms'] = speed_dict['inference']
                        run_data['fps'] = 1000.0 / speed_dict['inference'] if speed_dict['inference'] > 0 else None
                        
                # Per-class metrics (HRIPCB has 6 classes)
                class_names = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spurious_copper', 'Spur']
                for i, class_name in enumerate(class_names):
                    class_map_key = f'metrics/mAP50(B)_{i}'
                    if class_map_key in summary:
                        run_data[f'{class_name}_mAP50'] = summary[class_map_key]
                        
                # Extract phase and experiment type from tags
                phase = 'Unknown'
                experiment_type = 'Unknown'
                
                for tag in run.tags:
                    if 'phase_1' in tag:
                        phase = 'Phase 1: Baselines'
                    elif 'phase_2' in tag:
                        phase = 'Phase 2: Attention'
                    elif 'phase_3' in tag:
                        phase = 'Phase 3: Loss/Resolution'
                        
                    if 'model_scaling' in tag:
                        experiment_type = 'Model Scaling'
                    elif 'attention_study' in tag:
                        experiment_type = 'Attention Study'
                    elif 'architecture_study' in tag:
                        experiment_type = 'Architecture Study'
                    elif 'resolution_study' in tag:
                        experiment_type = 'Resolution Study'
                        
                run_data['phase'] = phase
                run_data['experiment_type'] = experiment_type
                
                # Data quality check
                if metrics_found < 3:  # At least 3 key metrics should be present
                    print(f"   ⚠️  Run {run.name}: Only {metrics_found} metrics found")
                
                runs_data.append(run_data)
                
            except Exception as e:
                print(f"   ❌ Error processing run {run.name}: {e}")
                failed_runs.append(run.name)
                continue
        
        # Create DataFrame
        df = pd.DataFrame(runs_data)
        
        if len(df) == 0:
            print("❌ No valid run data extracted")
            return pd.DataFrame()
        
        # Data cleaning and preprocessing
        df = clean_enhanced_dataframe(df)
        
        # Report results
        print(f"\\n📊 DATA EXTRACTION SUMMARY:")
        print(f"   ✅ Successfully processed: {len(df)} runs")
        print(f"   ❌ Failed to process: {len(failed_runs)} runs")
        
        if failed_runs:
            print(f"   Failed runs: {', '.join(failed_runs[:5])}{'...' if len(failed_runs) > 5 else ''}")
        
        # Data quality report
        key_metrics = ['map50', 'map50_95', 'precision', 'recall', 'f1_score', 'gflops']
        print(f"\\n🔍 DATA QUALITY REPORT:")
        for metric in key_metrics:
            if metric in df.columns:
                valid_count = df[metric].notna().sum()
                percentage = (valid_count / len(df)) * 100
                status = "✅" if percentage >= 80 else "⚠️ " if percentage >= 50 else "❌"
                print(f"   {status} {metric}: {valid_count}/{len(df)} ({percentage:.1f}%)")
        
        return df
        
    except Exception as e:
        print(f"❌ Critical error during data fetching: {e}")
        return pd.DataFrame()


def clean_enhanced_dataframe(df):
    """
    Enhanced data cleaning and preprocessing with better validation.
    """
    print("🧹 Cleaning and preprocessing data...")
    
    # Convert numeric columns with better error handling
    numeric_columns = [
        'map50', 'map50_95', 'precision', 'recall', 'f1_score',
        'box_loss', 'cls_loss', 'dfl_loss', 'val_box_loss', 'val_cls_loss', 'val_dfl_loss',
        'total_parameters', 'trainable_parameters', 'training_time_seconds',
        'batch_size', 'image_size', 'epochs', 'learning_rate', 'weight_decay',
        'box_weight', 'cls_weight', 'dfl_weight', 'inference_time_ms', 'fps',
        'duration_seconds', 'gflops'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert duration to hours
    if 'duration_seconds' in df.columns:
        df['duration_hours'] = df['duration_seconds'] / 3600
        
    if 'training_time_seconds' in df.columns:
        df['training_time_hours'] = df['training_time_seconds'] / 3600
    
    # Create model variant column
    df['model_variant'] = df.apply(create_model_variant, axis=1)
    
    # Calculate efficiency metrics with proper validation
    if 'map50' in df.columns and 'training_time_hours' in df.columns:
        valid_mask = (df['map50'].notna()) & (df['training_time_hours'].notna()) & (df['training_time_hours'] > 0)
        df.loc[valid_mask, 'training_efficiency'] = df.loc[valid_mask, 'map50'] / df.loc[valid_mask, 'training_time_hours']
        
    if 'map50' in df.columns and 'total_parameters' in df.columns:
        valid_mask = (df['map50'].notna()) & (df['total_parameters'].notna()) & (df['total_parameters'] > 0)
        df.loc[valid_mask, 'parameter_efficiency'] = df.loc[valid_mask, 'map50'] / (df.loc[valid_mask, 'total_parameters'] / 1e6)  # per million params
        
    # Estimate model size in MB (rough approximation)
    if 'total_parameters' in df.columns:
        valid_mask = df['total_parameters'].notna() & (df['total_parameters'] > 0)
        df.loc[valid_mask, 'model_size_mb'] = (df.loc[valid_mask, 'total_parameters'] * 4) / (1024 * 1024)  # Assuming FP32
    
    # Add quality score based on data completeness
    key_metrics = ['map50', 'precision', 'recall', 'f1_score']
    df['data_quality_score'] = df[key_metrics].notna().sum(axis=1) / len(key_metrics)
    
    print(f"✅ Data cleaning complete")
    print(f"   📊 Rows processed: {len(df)}")
    print(f"   🎯 Average data quality: {df['data_quality_score'].mean():.2f}")
    
    return df


print("✅ Enhanced data fetching functions defined")

## 4. Enhanced Data Fetching with GFLOPs and Media Support

In [ ]:
# Fetch data from WandB with enhanced validation
if WANDB_AUTHENTICATED:
    df = fetch_wandb_runs(WANDB_PROJECT, WANDB_ENTITY)
    
    if len(df) > 0:
        # Display basic information about the dataset
        print(f"\n📊 DATASET OVERVIEW:")
        print(f"   Total completed experiments: {len(df)}")
        print(f"   Unique model types: {df['model_type'].nunique()}")
        print(f"   Unique attention mechanisms: {df['attention_mechanism'].nunique()}")
        print(f"   Date range: {df['created_at'].min()} to {df['created_at'].max()}")
        
        # Check data quality
        key_metrics = ['map50', 'map50_95', 'precision', 'recall', 'f1_score', 'gflops']
        print(f"\n🔍 FINAL DATA QUALITY CHECK:")
        for metric in key_metrics:
            if metric in df.columns:
                valid_count = df[metric].notna().sum()
                percentage = (valid_count / len(df)) * 100
                status = "✅" if percentage >= 80 else "⚠️ " if percentage >= 50 else "❌"
                print(f"   {status} {metric}: {valid_count}/{len(df)} ({percentage:.1f}%)")
        
        # Display sample data with enhanced columns
        print(f"\n📋 SAMPLE DATA (Enhanced with GFLOPs):")
        display_columns = ['run_name', 'model_variant', 'map50', 'map50_95', 'f1_score', 'gflops', 'fps', 'training_time_hours']
        available_columns = [col for col in display_columns if col in df.columns]
        sample_df = df[available_columns].head()
        
        # Round numeric columns for display
        numeric_cols = sample_df.select_dtypes(include=[np.number]).columns
        sample_df[numeric_cols] = sample_df[numeric_cols].round(4)
        
        print(sample_df.to_string(index=False))
        
        # Show data readiness status
        if df['data_quality_score'].mean() >= 0.8:
            print(f"\n✅ DATA READY FOR ANALYSIS - High quality data available")
        elif df['data_quality_score'].mean() >= 0.6:
            print(f"\n⚠️  DATA PARTIALLY READY - Some metrics may be limited")
        else:
            print(f"\n❌ DATA QUALITY CONCERNS - Analysis may be limited")
            
    else:
        print(f"\n❌ NO DATA AVAILABLE FOR ANALYSIS")
        print("Please check your WandB project and ensure experiments have completed successfully")
else:
    print(f"\n❌ WANDB AUTHENTICATION REQUIRED")
    print("Please run the authentication cell above first")
    df = pd.DataFrame()  # Empty dataframe as fallback

## 3. WandB Authentication & API Validation

In [ ]:
def fetch_wandb_runs(project_name, entity=None):
    """
    Fetch all runs from WandB project and extract comprehensive metrics.
    
    Returns:
        pd.DataFrame: Comprehensive experiment results
    """
    print("🔄 Fetching runs from WandB...")
    
    # Initialize WandB API
    api = wandb.Api()
    
    # Get project path
    project_path = f"{entity}/{project_name}" if entity else project_name
    runs = api.runs(project_path)
    
    print(f"📥 Found {len(runs)} runs")
    
    # Extract run data
    runs_data = []
    
    for i, run in enumerate(runs):
        if i % 5 == 0:  # Progress indicator
            print(f"   Processing run {i+1}/{len(runs)}...")
            
        try:
            # Basic run information
            run_data = {
                'run_id': run.id,
                'run_name': run.name,
                'state': run.state,
                'created_at': run.created_at,
                'duration_seconds': run._attrs.get('runtime', 0),
                'tags': run.tags,
                'notes': run.notes,
                'url': run.url
            }
            
            # Extract configuration
            config = run.config
            
            # Model configuration
            model_config = config.get('model_config', {})
            run_data.update({
                'model_type': model_config.get('type', 'unknown'),
                'attention_mechanism': model_config.get('attention_mechanism', 'none'),
                'model_config_path': model_config.get('config_path', ''),
                'pretrained': model_config.get('pretrained', True)
            })
            
            # Training configuration  
            training_config = config.get('training_config', {})
            run_data.update({
                'batch_size': training_config.get('batch', 'unknown'),
                'image_size': training_config.get('imgsz', 640),
                'epochs': training_config.get('epochs', 'unknown'),
                'optimizer': training_config.get('optimizer', 'unknown'),
                'learning_rate': training_config.get('lr0', 'unknown'),
                'weight_decay': training_config.get('weight_decay', 'unknown'),
                'amp': training_config.get('amp', True),
                'cache': training_config.get('cache', True)
            })
            
            # Loss configuration
            loss_config = training_config.get('loss', {})
            run_data.update({
                'loss_type': loss_config.get('type', 'standard'),
                'box_weight': loss_config.get('box_weight', 7.5),
                'cls_weight': loss_config.get('cls_weight', 0.5),
                'dfl_weight': loss_config.get('dfl_weight', 1.5)
            })
            
            # Data configuration
            data_config = config.get('data_config', {})
            run_data['num_classes'] = data_config.get('num_classes', 6)
            
            # Extract final metrics from summary
            summary = run.summary
            
            # Performance metrics with multiple possible keys
            metrics_mapping = {
                'map50': ['metrics/mAP50', 'metrics/mAP50(B)', 'val/mAP50', 'mAP50'],
                'map50_95': ['metrics/mAP50-95', 'metrics/mAP50-95(B)', 'val/mAP50-95', 'mAP50-95'],
                'precision': ['metrics/precision(B)', 'metrics/precision', 'val/precision', 'precision'],
                'recall': ['metrics/recall(B)', 'metrics/recall', 'val/recall', 'recall'],
                'f1_score': ['metrics/F1(B)', 'metrics/F1', 'val/F1', 'f1'],
                'box_loss': ['train/box_loss', 'box_loss'],
                'cls_loss': ['train/cls_loss', 'cls_loss'], 
                'dfl_loss': ['train/dfl_loss', 'dfl_loss'],
                'val_box_loss': ['val/box_loss'],
                'val_cls_loss': ['val/cls_loss'],
                'val_dfl_loss': ['val/dfl_loss'],
                'total_parameters': ['model/total_parameters'],
                'trainable_parameters': ['model/trainable_parameters'],
                'training_time_seconds': ['final/training_time_seconds', 'training_time']
            }
            
            # Extract metrics with fallback
            for metric_name, possible_keys in metrics_mapping.items():
                value = None
                for key in possible_keys:
                    if key in summary:
                        value = summary[key]
                        break
                run_data[metric_name] = value
                
            # Speed metrics (if available)
            if 'speed/inference' in summary:
                run_data['inference_time_ms'] = summary['speed/inference']
                run_data['fps'] = 1000.0 / summary['speed/inference'] if summary['speed/inference'] > 0 else None
            elif 'speed' in summary and isinstance(summary['speed'], dict):
                speed_dict = summary['speed']
                if 'inference' in speed_dict:
                    run_data['inference_time_ms'] = speed_dict['inference']
                    run_data['fps'] = 1000.0 / speed_dict['inference'] if speed_dict['inference'] > 0 else None
                    
            # Per-class metrics (HRIPCB has 6 classes)
            class_names = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spurious_copper', 'Spur']
            for i, class_name in enumerate(class_names):
                class_map_key = f'metrics/mAP50(B)_{i}'
                if class_map_key in summary:
                    run_data[f'{class_name}_mAP50'] = summary[class_map_key]
                    
            # Extract phase and experiment type from tags
            phase = 'Unknown'
            experiment_type = 'Unknown'
            
            for tag in run.tags:
                if 'phase_1' in tag:
                    phase = 'Phase 1: Baselines'
                elif 'phase_2' in tag:
                    phase = 'Phase 2: Attention'
                elif 'phase_3' in tag:
                    phase = 'Phase 3: Loss/Resolution'
                    
                if 'model_scaling' in tag:
                    experiment_type = 'Model Scaling'
                elif 'attention_study' in tag:
                    experiment_type = 'Attention Study'
                elif 'architecture_study' in tag:
                    experiment_type = 'Architecture Study'
                elif 'resolution_study' in tag:
                    experiment_type = 'Resolution Study'
                    
            run_data['phase'] = phase
            run_data['experiment_type'] = experiment_type
            
            runs_data.append(run_data)
            
        except Exception as e:
            print(f"   ⚠️  Error processing run {run.name}: {e}")
            continue
    
    # Create DataFrame
    df = pd.DataFrame(runs_data)
    
    # Data cleaning and preprocessing
    df = clean_dataframe(df)
    
    print(f"✅ Successfully processed {len(df)} runs")
    print(f"   📊 Completed: {len(df[df['state'] == 'finished'])}")
    print(f"   🔄 Running: {len(df[df['state'] == 'running'])}")
    print(f"   ❌ Failed: {len(df[df['state'] == 'failed'])}")
    
    return df


def clean_dataframe(df):
    """
    Clean and preprocess the dataframe.
    """
    # Convert numeric columns
    numeric_columns = [
        'map50', 'map50_95', 'precision', 'recall', 'f1_score',
        'box_loss', 'cls_loss', 'dfl_loss', 'val_box_loss', 'val_cls_loss', 'val_dfl_loss',
        'total_parameters', 'trainable_parameters', 'training_time_seconds',
        'batch_size', 'image_size', 'epochs', 'learning_rate', 'weight_decay',
        'box_weight', 'cls_weight', 'dfl_weight', 'inference_time_ms', 'fps',
        'duration_seconds'
    ]
    
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Convert duration to hours
    if 'duration_seconds' in df.columns:
        df['duration_hours'] = df['duration_seconds'] / 3600
        
    if 'training_time_seconds' in df.columns:
        df['training_time_hours'] = df['training_time_seconds'] / 3600
    
    # Create model variant column
    df['model_variant'] = df.apply(create_model_variant, axis=1)
    
    # Calculate efficiency metrics
    if 'map50' in df.columns and 'training_time_hours' in df.columns:
        df['training_efficiency'] = df['map50'] / df['training_time_hours']
        
    if 'map50' in df.columns and 'total_parameters' in df.columns:
        df['parameter_efficiency'] = df['map50'] / (df['total_parameters'] / 1e6)  # per million params
        
    # Estimate model size in MB (rough approximation)
    if 'total_parameters' in df.columns:
        df['model_size_mb'] = (df['total_parameters'] * 4) / (1024 * 1024)  # Assuming FP32
    
    # Filter only completed runs for analysis
    df_completed = df[df['state'] == 'finished'].copy()
    
    return df_completed


def create_model_variant(row):
    """
    Create a descriptive model variant name.
    """
    model_type = row.get('model_type', 'unknown')
    attention = row.get('attention_mechanism', 'none')
    image_size = row.get('image_size', 640)
    
    # Clean up model type
    model_str = model_type.upper() if pd.notna(model_type) else 'UNKNOWN'
    
    # Clean up attention mechanism
    if pd.isna(attention) or attention == 'none' or attention == 'unknown':
        attention_str = ""
    else:
        attention_str = f" + {attention.upper()}"
    
    # Add resolution if different from standard
    if pd.notna(image_size) and image_size != 640:
        size_str = f" ({int(image_size)}px)"
    else:
        size_str = ""
    
    return f"{model_str}{attention_str}{size_str}"


print("✅ Data fetching functions defined")

## 4. Fetch Experimental Data

In [ ]:
# Generate summary statistics
if len(df) > 0:
    generate_summary_statistics(df)

# Create comprehensive comparison table with GFLOPs
print("\n" + "="*80)
print("📋 COMPREHENSIVE COMPARISON TABLE (Enhanced with GFLOPs)")
print("="*80)

if len(df) > 0:
    # Select key columns for comparison including GFLOPs
    comparison_columns = [
        'model_variant', 
        # Accuracy metrics
        'map50', 'map50_95', 'precision', 'recall', 'f1_score',
        # Efficiency metrics with GFLOPs
        'inference_time_ms', 'fps', 'gflops', 'total_parameters', 'model_size_mb',
        # Training metrics
        'training_time_hours'
    ]
    
    # Filter available columns
    available_comparison_columns = [col for col in comparison_columns if col in df.columns]
    
    # Create comparison table
    comparison_df = df[available_comparison_columns].copy()
    
    # Round numeric columns for display
    numeric_columns = comparison_df.select_dtypes(include=[np.number]).columns
    comparison_df[numeric_columns] = comparison_df[numeric_columns].round(4)
    
    # Sort by mAP@0.5 if available
    if 'map50' in comparison_df.columns:
        comparison_df = comparison_df.sort_values('map50', ascending=False)
    
    print("📊 ACCURACY & EFFICIENCY METRICS:")
    print("• Accuracy: mAP@0.5, mAP@0.5:0.95, Precision, Recall, F1")
    print("• Efficiency: Inference Time (ms), FPS, GFLOPs, Parameters, Model Size (MB)")
    print("-" * 80)
    print(comparison_df.to_string(index=False))
    
    # Create separate detailed efficiency table
    if all(col in df.columns for col in ['gflops', 'fps', 'model_size_mb']):
        print(f"\n📊 DETAILED EFFICIENCY ANALYSIS:")
        efficiency_df = df[['model_variant', 'map50', 'fps', 'gflops', 'model_size_mb', 'parameter_efficiency']].copy()
        efficiency_df = efficiency_df.dropna()
        
        if len(efficiency_df) > 0:
            # Add efficiency ratio (mAP per GFLOPs)
            efficiency_df['map_per_gflop'] = efficiency_df['map50'] / efficiency_df['gflops']
            efficiency_df = efficiency_df.round(4)
            efficiency_df = efficiency_df.sort_values('map_per_gflop', ascending=False)
            
            print("• Efficiency Ratio: mAP@0.5 per GFLOPs (higher is better)")
            print("-" * 60)
            print(efficiency_df[['model_variant', 'map50', 'gflops', 'map_per_gflop', 'fps', 'model_size_mb']].to_string(index=False))
    
    # Save to CSV
    if SAVE_PLOTS:
        csv_path = OUTPUT_DIR / "enhanced_experiment_comparison_table.csv"
        comparison_df.to_csv(csv_path, index=False)
        print(f"\n💾 Enhanced comparison table saved to: {csv_path}")
        
        # Also save efficiency analysis
        if 'efficiency_df' in locals() and len(efficiency_df) > 0:
            eff_csv_path = OUTPUT_DIR / "efficiency_analysis_table.csv"
            efficiency_df.to_csv(eff_csv_path, index=False)
            print(f"💾 Efficiency analysis saved to: {eff_csv_path}")
else:
    print("⚠️  No data available for comparison")
    
# Add media retrieval demonstration if data is available
if len(df) > 0 and WANDB_AUTHENTICATED:
    print(f"\n" + "="*80)
    print("🎨 ULTRALYTICS MEDIA RETRIEVAL DEMONSTRATION")
    print("="*80)
    
    try:
        # Demonstrate media retrieval for top 3 runs
        demonstrate_media_retrieval(df, n_runs=3)
    except Exception as e:
        print(f"⚠️  Media retrieval demonstration failed: {e}")
        print("This is normal if your experiments don't have logged media artifacts")
else:
    print(f"\n⚠️  Skipping media retrieval demonstration (no data or not authenticated)")

## 5. Data Analysis Functions

In [ ]:
def generate_summary_statistics(df):
    """
    Generate comprehensive summary statistics.
    """
    print("📊 PERFORMANCE SUMMARY STATISTICS")
    print("=" * 50)
    
    # Overall statistics
    metrics = ['map50', 'map50_95', 'precision', 'recall', 'f1_score']
    
    for metric in metrics:
        if metric in df.columns and not df[metric].isna().all():
            values = df[metric].dropna()
            print(f"\n{metric.upper()}:")
            print(f"   Mean: {values.mean():.4f} ± {values.std():.4f}")
            print(f"   Range: [{values.min():.4f}, {values.max():.4f}]")
            print(f"   Best model: {df.loc[values.idxmax(), 'model_variant']}")
    
    # Model comparison
    if 'model_type' in df.columns and 'map50' in df.columns:
        print(f"\n📈 MODEL TYPE COMPARISON (mAP@0.5):")
        model_stats = df.groupby('model_type')['map50'].agg(['count', 'mean', 'std']).round(4)
        print(model_stats)
    
    # Attention mechanism comparison
    if 'attention_mechanism' in df.columns and 'map50' in df.columns:
        print(f"\n🎯 ATTENTION MECHANISM COMPARISON (mAP@0.5):")
        attention_stats = df.groupby('attention_mechanism')['map50'].agg(['count', 'mean', 'std']).round(4)
        print(attention_stats)
    
    # Training efficiency
    if 'training_time_hours' in df.columns:
        total_training_time = df['training_time_hours'].sum()
        print(f"\n⏱️  TRAINING EFFICIENCY:")
        print(f"   Total training time: {total_training_time:.1f} hours")
        print(f"   Average per experiment: {df['training_time_hours'].mean():.1f} hours")
        
        if 'training_efficiency' in df.columns:
            best_efficiency_idx = df['training_efficiency'].idxmax()
            best_efficient_model = df.loc[best_efficiency_idx, 'model_variant']
            best_efficiency_score = df.loc[best_efficiency_idx, 'training_efficiency']
            print(f"   Most efficient: {best_efficient_model} ({best_efficiency_score:.4f} mAP/hour)")


def identify_pareto_optimal(df, x_metric, y_metric, maximize_x=True, maximize_y=True):
    """
    Identify Pareto optimal points for trade-off analysis.
    
    Args:
        df: DataFrame with experimental results
        x_metric: Column name for x-axis metric
        y_metric: Column name for y-axis metric  
        maximize_x: Whether to maximize x_metric (True) or minimize (False)
        maximize_y: Whether to maximize y_metric (True) or minimize (False)
        
    Returns:
        pd.DataFrame: Pareto optimal points
    """
    # Filter valid data
    valid_data = df.dropna(subset=[x_metric, y_metric]).copy()
    
    if len(valid_data) == 0:
        return pd.DataFrame()
    
    # Adjust for minimization (convert to maximization problem)
    x_values = valid_data[x_metric].values * (1 if maximize_x else -1)
    y_values = valid_data[y_metric].values * (1 if maximize_y else -1)
    
    # Find Pareto optimal points
    pareto_mask = np.zeros(len(valid_data), dtype=bool)
    
    for i in range(len(valid_data)):
        is_dominated = False
        for j in range(len(valid_data)):
            if i != j:
                # Check if point i is dominated by point j
                if (x_values[j] >= x_values[i] and y_values[j] >= y_values[i] and 
                    (x_values[j] > x_values[i] or y_values[j] > y_values[i])):
                    is_dominated = True
                    break
        
        if not is_dominated:
            pareto_mask[i] = True
    
    pareto_optimal = valid_data[pareto_mask].copy()
    
    # Sort by x_metric for plotting
    pareto_optimal = pareto_optimal.sort_values(x_metric, ascending=not maximize_x)
    
    return pareto_optimal


def filter_edge_suitable_models(df):
    """
    Filter models suitable for edge deployment based on constraints.
    """
    edge_suitable = df.copy()
    
    # Apply constraints
    constraints_applied = []
    
    if 'model_size_mb' in df.columns:
        before_count = len(edge_suitable)
        edge_suitable = edge_suitable[edge_suitable['model_size_mb'] <= EDGE_CONSTRAINTS['max_model_size_mb']]
        constraints_applied.append(f"Model size ≤ {EDGE_CONSTRAINTS['max_model_size_mb']}MB: {before_count} → {len(edge_suitable)}")
    
    if 'fps' in df.columns:
        before_count = len(edge_suitable)
        edge_suitable = edge_suitable[edge_suitable['fps'] >= EDGE_CONSTRAINTS['min_fps']]
        constraints_applied.append(f"FPS ≥ {EDGE_CONSTRAINTS['min_fps']}: {before_count} → {len(edge_suitable)}")
    
    if 'map50' in df.columns:
        before_count = len(edge_suitable)
        edge_suitable = edge_suitable[edge_suitable['map50'] >= EDGE_CONSTRAINTS['min_map50']]
        constraints_applied.append(f"mAP@0.5 ≥ {EDGE_CONSTRAINTS['min_map50']}: {before_count} → {len(edge_suitable)}")
    
    if 'inference_time_ms' in df.columns:
        before_count = len(edge_suitable)
        edge_suitable = edge_suitable[edge_suitable['inference_time_ms'] <= EDGE_CONSTRAINTS['max_inference_time_ms']]
        constraints_applied.append(f"Inference time ≤ {EDGE_CONSTRAINTS['max_inference_time_ms']}ms: {before_count} → {len(edge_suitable)}")
    
    print(f"🔍 Edge Deployment Filtering:")
    print(f"   Original models: {len(df)}")
    for constraint in constraints_applied:
        print(f"   {constraint}")
    print(f"   ✅ Edge-suitable models: {len(edge_suitable)}")
    
    return edge_suitable


print("✅ Analysis functions defined")

## 6. Comprehensive Performance Analysis

In [ ]:
# Generate summary statistics
generate_summary_statistics(df)

# Create comprehensive comparison table
print("\n" + "="*80)
print("📋 COMPREHENSIVE COMPARISON TABLE")
print("="*80)

if len(df) > 0:
    # Select key columns for comparison
    comparison_columns = [
        'model_variant', 'map50', 'map50_95', 'precision', 'recall', 'f1_score',
        'training_time_hours', 'total_parameters', 'model_size_mb',
        'fps', 'inference_time_ms'
    ]
    
    # Filter available columns
    available_comparison_columns = [col for col in comparison_columns if col in df.columns]
    
    # Create comparison table
    comparison_df = df[available_comparison_columns].copy()
    
    # Round numeric columns for display
    numeric_columns = comparison_df.select_dtypes(include=[np.number]).columns
    comparison_df[numeric_columns] = comparison_df[numeric_columns].round(4)
    
    # Sort by mAP@0.5 if available
    if 'map50' in comparison_df.columns:
        comparison_df = comparison_df.sort_values('map50', ascending=False)
    
    print(comparison_df.to_string(index=False))
    
    # Save to CSV
    if SAVE_PLOTS:
        csv_path = OUTPUT_DIR / "experiment_comparison_table.csv"
        comparison_df.to_csv(csv_path, index=False)
        print(f"\n💾 Comparison table saved to: {csv_path}")
else:
    print("⚠️  No data available for comparison")

## 7. Performance Visualizations

In [ ]:
def create_performance_comparison_plots(df):
    """
    Create comprehensive performance comparison visualizations.
    """
    if len(df) == 0:
        print("⚠️  No data available for plotting")
        return
    
    # Set up the plotting grid
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('PCB Defect Detection: Performance Comparison', fontsize=16, fontweight='bold')
    
    # 1. Model Type Comparison (Box Plot)
    if 'model_type' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['model_type', 'map50'])
        if len(valid_data) > 0:
            sns.boxplot(data=valid_data, x='model_type', y='map50', ax=axes[0,0])
            axes[0,0].set_title('Model Type Performance Comparison')
            axes[0,0].set_xlabel('Model Type')
            axes[0,0].set_ylabel('mAP@0.5')
            axes[0,0].tick_params(axis='x', rotation=45)
    
    # 2. Attention Mechanism Comparison
    if 'attention_mechanism' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['attention_mechanism', 'map50'])
        if len(valid_data) > 0:
            sns.boxplot(data=valid_data, x='attention_mechanism', y='map50', ax=axes[0,1])
            axes[0,1].set_title('Attention Mechanism Comparison')
            axes[0,1].set_xlabel('Attention Mechanism')
            axes[0,1].set_ylabel('mAP@0.5')
            axes[0,1].tick_params(axis='x', rotation=45)
    
    # 3. Resolution Study
    if 'image_size' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['image_size', 'map50'])
        if len(valid_data) > 0:
            sns.scatterplot(data=valid_data, x='image_size', y='map50', 
                          hue='model_type', size='total_parameters', 
                          sizes=(50, 200), alpha=0.7, ax=axes[1,0])
            axes[1,0].set_title('Resolution vs Performance')
            axes[1,0].set_xlabel('Image Size (pixels)')
            axes[1,0].set_ylabel('mAP@0.5')
    
    # 4. Training Efficiency
    if 'training_time_hours' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['training_time_hours', 'map50'])
        if len(valid_data) > 0:
            sns.scatterplot(data=valid_data, x='training_time_hours', y='map50',
                          hue='model_type', size='total_parameters',
                          sizes=(50, 200), alpha=0.7, ax=axes[1,1])
            axes[1,1].set_title('Training Time vs Performance')
            axes[1,1].set_xlabel('Training Time (hours)')
            axes[1,1].set_ylabel('mAP@0.5')
    
    plt.tight_layout()
    
    if SAVE_PLOTS:
        plt.savefig(OUTPUT_DIR / 'performance_comparison.png', dpi=300, bbox_inches='tight')
        print(f"💾 Performance comparison plot saved")
    
    plt.show()


def create_metrics_correlation_heatmap(df):
    """
    Create correlation heatmap between different metrics.
    """
    metrics_columns = ['map50', 'map50_95', 'precision', 'recall', 'f1_score',
                      'training_time_hours', 'total_parameters']
    
    available_metrics = [col for col in metrics_columns if col in df.columns and not df[col].isna().all()]
    
    if len(available_metrics) < 2:
        print("⚠️  Insufficient metrics for correlation analysis")
        return
    
    plt.figure(figsize=(10, 8))
    
    # Calculate correlation matrix
    corr_matrix = df[available_metrics].corr()
    
    # Create heatmap
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
    
    plt.title('Metrics Correlation Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    if SAVE_PLOTS:
        plt.savefig(OUTPUT_DIR / 'metrics_correlation.png', dpi=300, bbox_inches='tight')
        print(f"💾 Correlation heatmap saved")
    
    plt.show()


# Create visualizations
print("📊 Creating performance visualizations...")
create_performance_comparison_plots(df)
create_metrics_correlation_heatmap(df)

## 8. Pareto Analysis for Edge Deployment

In [ ]:
def create_pareto_analysis(df):
    """
    Create comprehensive Pareto analysis for edge deployment optimization.
    """
    print("🎯 PARETO ANALYSIS FOR EDGE DEPLOYMENT")
    print("=" * 60)
    
    if len(df) == 0:
        print("⚠️  No data available for Pareto analysis")
        return
    
    # Create subplot figure
    fig = plt.figure(figsize=(20, 12))
    
    # 1. mAP vs FPS (Performance vs Speed)
    if 'fps' in df.columns and 'map50' in df.columns:
        ax1 = plt.subplot(2, 3, 1)
        
        valid_data = df.dropna(subset=['fps', 'map50'])
        if len(valid_data) > 0:
            # Find Pareto optimal points
            pareto_points = identify_pareto_optimal(valid_data, 'fps', 'map50', 
                                                  maximize_x=True, maximize_y=True)
            
            # Plot all points
            scatter = plt.scatter(valid_data['fps'], valid_data['map50'], 
                                c='lightblue', alpha=0.6, s=50, label='All Models')
            
            # Plot Pareto optimal points
            if len(pareto_points) > 0:
                plt.scatter(pareto_points['fps'], pareto_points['map50'], 
                          c='red', s=100, label='Pareto Optimal', marker='*')
                
                # Connect Pareto points
                pareto_sorted = pareto_points.sort_values('fps')
                plt.plot(pareto_sorted['fps'], pareto_sorted['map50'], 
                        'r--', alpha=0.7, label='Pareto Front')
                
                # Annotate Pareto points
                for _, row in pareto_points.iterrows():
                    plt.annotate(row['model_variant'], 
                               (row['fps'], row['map50']),
                               xytext=(5, 5), textcoords='offset points', 
                               fontsize=8, ha='left')
            
            plt.xlabel('FPS (Inference Speed)')
            plt.ylabel('mAP@0.5 (Performance)')
            plt.title('Performance vs Speed Trade-off')
            plt.legend()
            plt.grid(True, alpha=0.3)
    
    # 2. mAP vs Model Size (Performance vs Efficiency)
    if 'model_size_mb' in df.columns and 'map50' in df.columns:
        ax2 = plt.subplot(2, 3, 2)
        
        valid_data = df.dropna(subset=['model_size_mb', 'map50'])
        if len(valid_data) > 0:
            # Find Pareto optimal points (minimize size, maximize performance)
            pareto_points = identify_pareto_optimal(valid_data, 'model_size_mb', 'map50',
                                                  maximize_x=False, maximize_y=True)
            
            # Plot all points
            plt.scatter(valid_data['model_size_mb'], valid_data['map50'],
                       c='lightgreen', alpha=0.6, s=50, label='All Models')
            
            # Plot Pareto optimal points
            if len(pareto_points) > 0:
                plt.scatter(pareto_points['model_size_mb'], pareto_points['map50'],
                          c='red', s=100, label='Pareto Optimal', marker='*')
                
                # Connect Pareto points
                pareto_sorted = pareto_points.sort_values('model_size_mb')
                plt.plot(pareto_sorted['model_size_mb'], pareto_sorted['map50'],
                        'r--', alpha=0.7, label='Pareto Front')
                
                # Annotate Pareto points
                for _, row in pareto_points.iterrows():
                    plt.annotate(row['model_variant'],
                               (row['model_size_mb'], row['map50']),
                               xytext=(5, 5), textcoords='offset points',
                               fontsize=8, ha='left')
            
            plt.xlabel('Model Size (MB)')
            plt.ylabel('mAP@0.5 (Performance)')
            plt.title('Performance vs Model Size Trade-off')
            plt.legend()
            plt.grid(True, alpha=0.3)
    
    # 3. Training Time vs Performance
    if 'training_time_hours' in df.columns and 'map50' in df.columns:
        ax3 = plt.subplot(2, 3, 3)
        
        valid_data = df.dropna(subset=['training_time_hours', 'map50'])
        if len(valid_data) > 0:
            # Color by model type
            if 'model_type' in df.columns:
                for model_type in valid_data['model_type'].unique():
                    model_data = valid_data[valid_data['model_type'] == model_type]
                    plt.scatter(model_data['training_time_hours'], model_data['map50'],
                              label=model_type, alpha=0.7, s=60)
            else:
                plt.scatter(valid_data['training_time_hours'], valid_data['map50'],
                          alpha=0.7, s=60)
            
            plt.xlabel('Training Time (hours)')
            plt.ylabel('mAP@0.5 (Performance)')
            plt.title('Training Efficiency Analysis')
            if 'model_type' in df.columns:
                plt.legend()
            plt.grid(True, alpha=0.3)
    
    # 4. 3D Performance-Speed-Size Plot
    if all(col in df.columns for col in ['fps', 'map50', 'model_size_mb']):
        ax4 = plt.subplot(2, 3, 4, projection='3d')
        
        valid_data = df.dropna(subset=['fps', 'map50', 'model_size_mb'])
        if len(valid_data) > 0:
            scatter = ax4.scatter(valid_data['fps'], valid_data['model_size_mb'], valid_data['map50'],
                                c=valid_data['map50'], cmap='viridis', s=60, alpha=0.7)
            
            ax4.set_xlabel('FPS')
            ax4.set_ylabel('Model Size (MB)')
            ax4.set_zlabel('mAP@0.5')
            ax4.set_title('3D Performance Trade-off')
            
            # Add colorbar
            plt.colorbar(scatter, ax=ax4, shrink=0.5, label='mAP@0.5')
    
    # 5. Edge Deployment Constraints Visualization
    ax5 = plt.subplot(2, 3, 5)
    
    # Filter edge-suitable models
    edge_suitable = filter_edge_suitable_models(df)
    
    if 'fps' in df.columns and 'map50' in df.columns:
        valid_all = df.dropna(subset=['fps', 'map50'])
        valid_edge = edge_suitable.dropna(subset=['fps', 'map50']) if len(edge_suitable) > 0 else pd.DataFrame()
        
        if len(valid_all) > 0:
            # Plot all models
            plt.scatter(valid_all['fps'], valid_all['map50'], 
                       c='lightgray', alpha=0.5, s=50, label='All Models')
            
            # Plot edge-suitable models
            if len(valid_edge) > 0:
                plt.scatter(valid_edge['fps'], valid_edge['map50'],
                          c='green', s=80, alpha=0.8, label='Edge Suitable')
                
                # Annotate edge-suitable models
                for _, row in valid_edge.iterrows():
                    plt.annotate(row['model_variant'],
                               (row['fps'], row['map50']),
                               xytext=(5, 5), textcoords='offset points',
                               fontsize=8, ha='left')
            
            # Add constraint lines
            plt.axhline(y=EDGE_CONSTRAINTS['min_map50'], color='red', 
                       linestyle='--', alpha=0.7, label=f"Min mAP@0.5 ({EDGE_CONSTRAINTS['min_map50']})")
            plt.axvline(x=EDGE_CONSTRAINTS['min_fps'], color='blue', 
                       linestyle='--', alpha=0.7, label=f"Min FPS ({EDGE_CONSTRAINTS['min_fps']})")
            
            plt.xlabel('FPS (Inference Speed)')
            plt.ylabel('mAP@0.5 (Performance)')
            plt.title('Edge Deployment Suitability')
            plt.legend()
            plt.grid(True, alpha=0.3)
    
    # 6. Model Efficiency Summary
    ax6 = plt.subplot(2, 3, 6)
    
    if 'parameter_efficiency' in df.columns and 'training_efficiency' in df.columns:
        valid_data = df.dropna(subset=['parameter_efficiency', 'training_efficiency'])
        if len(valid_data) > 0:
            scatter = plt.scatter(valid_data['parameter_efficiency'], valid_data['training_efficiency'],
                                c=valid_data['map50'] if 'map50' in df.columns else 'blue',
                                s=80, alpha=0.7, cmap='viridis')
            
            # Annotate points
            for _, row in valid_data.iterrows():
                plt.annotate(row['model_variant'],
                           (row['parameter_efficiency'], row['training_efficiency']),
                           xytext=(5, 5), textcoords='offset points',
                           fontsize=8, ha='left')
            
            plt.xlabel('Parameter Efficiency (mAP/M params)')
            plt.ylabel('Training Efficiency (mAP/hour)')
            plt.title('Model Efficiency Comparison')
            
            if 'map50' in df.columns:
                plt.colorbar(scatter, label='mAP@0.5')
            
            plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if SAVE_PLOTS:
        plt.savefig(OUTPUT_DIR / 'pareto_analysis.png', dpi=300, bbox_inches='tight')
        print(f"💾 Pareto analysis plot saved")
    
    plt.show()
    
    # Print Pareto optimal models summary
    if 'fps' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['fps', 'map50'])
        if len(valid_data) > 0:
            pareto_points = identify_pareto_optimal(valid_data, 'fps', 'map50')
            
            if len(pareto_points) > 0:
                print(f"\n🏆 PARETO OPTIMAL MODELS (Performance vs Speed):")
                pareto_display = pareto_points[['model_variant', 'map50', 'fps', 'model_size_mb']].round(4)
                print(pareto_display.to_string(index=False))


# Run Pareto Analysis
create_pareto_analysis(df)

## 9. Interactive Plotly Visualizations (Optional)

In [ ]:
if PLOTLY_AVAILABLE and len(df) > 0:
    print("🎨 Creating interactive Plotly visualizations...")
    
    # Interactive 3D Performance Trade-off Plot
    if all(col in df.columns for col in ['fps', 'map50', 'model_size_mb', 'model_variant']):
        valid_data = df.dropna(subset=['fps', 'map50', 'model_size_mb'])
        
        if len(valid_data) > 0:
            fig = go.Figure(data=[go.Scatter3d(
                x=valid_data['fps'],
                y=valid_data['model_size_mb'],
                z=valid_data['map50'],
                mode='markers',
                marker=dict(
                    size=8,
                    color=valid_data['map50'],
                    colorscale='Viridis',
                    showscale=True,
                    colorbar=dict(title="mAP@0.5")
                ),
                text=valid_data['model_variant'],
                hovertemplate=
                '<b>%{text}</b><br>' +
                'FPS: %{x}<br>' +
                'Model Size: %{y} MB<br>' +
                'mAP@0.5: %{z}<br>' +
                '<extra></extra>'
            )])
            
            fig.update_layout(
                title='Interactive 3D Performance Trade-off Analysis',
                scene=dict(
                    xaxis_title='FPS (Inference Speed)',
                    yaxis_title='Model Size (MB)',
                    zaxis_title='mAP@0.5 (Performance)'
                ),
                width=800,
                height=600
            )
            
            fig.show()
            
            if SAVE_PLOTS:
                fig.write_html(str(OUTPUT_DIR / 'interactive_3d_analysis.html'))
                print(f"💾 Interactive 3D plot saved as HTML")
    
    # Interactive Pareto Plot
    if 'fps' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['fps', 'map50'])
        pareto_points = identify_pareto_optimal(valid_data, 'fps', 'map50')
        
        if len(valid_data) > 0:
            fig = go.Figure()
            
            # Add all points
            fig.add_trace(go.Scatter(
                x=valid_data['fps'],
                y=valid_data['map50'],
                mode='markers',
                name='All Models',
                marker=dict(color='lightblue', size=8, opacity=0.6),
                text=valid_data['model_variant'],
                hovertemplate='<b>%{text}</b><br>FPS: %{x}<br>mAP@0.5: %{y}<extra></extra>'
            ))
            
            # Add Pareto optimal points
            if len(pareto_points) > 0:
                fig.add_trace(go.Scatter(
                    x=pareto_points['fps'],
                    y=pareto_points['map50'],
                    mode='markers+lines',
                    name='Pareto Optimal',
                    marker=dict(color='red', size=12, symbol='star'),
                    line=dict(color='red', dash='dash'),
                    text=pareto_points['model_variant'],
                    hovertemplate='<b>%{text}</b><br>FPS: %{x}<br>mAP@0.5: %{y}<br><b>Pareto Optimal</b><extra></extra>'
                ))
            
            # Add constraint lines
            fig.add_hline(y=EDGE_CONSTRAINTS['min_map50'], line_dash="dash", 
                         line_color="red", annotation_text=f"Min mAP@0.5 ({EDGE_CONSTRAINTS['min_map50']})")
            fig.add_vline(x=EDGE_CONSTRAINTS['min_fps'], line_dash="dash", 
                         line_color="blue", annotation_text=f"Min FPS ({EDGE_CONSTRAINTS['min_fps']})")
            
            fig.update_layout(
                title='Interactive Performance vs Speed Trade-off',
                xaxis_title='FPS (Inference Speed)',
                yaxis_title='mAP@0.5 (Performance)',
                width=800,
                height=600,
                showlegend=True
            )
            
            fig.show()
            
            if SAVE_PLOTS:
                fig.write_html(str(OUTPUT_DIR / 'interactive_pareto_plot.html'))
                print(f"💾 Interactive Pareto plot saved as HTML")
else:
    print("⚠️  Plotly not available or no data for interactive plots")

## 10. Model Recommendation System

In [ ]:
def recommend_models_for_deployment(df):
    """
    Recommend optimal models for different deployment scenarios.
    """
    print("🎯 MODEL RECOMMENDATIONS FOR DEPLOYMENT")
    print("=" * 60)
    
    if len(df) == 0:
        print("⚠️  No data available for recommendations")
        return
    
    recommendations = {}
    
    # 1. Best Overall Performance
    if 'map50' in df.columns:
        best_performance_idx = df['map50'].idxmax()
        best_performance_model = df.loc[best_performance_idx]
        
        recommendations['best_performance'] = {
            'model': best_performance_model['model_variant'],
            'map50': best_performance_model['map50'],
            'reasoning': 'Highest mAP@0.5 score - best for accuracy-critical applications'
        }
    
    # 2. Best Speed (Highest FPS)
    if 'fps' in df.columns:
        valid_fps_data = df.dropna(subset=['fps'])
        if len(valid_fps_data) > 0:
            best_speed_idx = valid_fps_data['fps'].idxmax()
            best_speed_model = valid_fps_data.loc[best_speed_idx]
            
            recommendations['best_speed'] = {
                'model': best_speed_model['model_variant'],
                'fps': best_speed_model['fps'],
                'map50': best_speed_model.get('map50', 'N/A'),
                'reasoning': 'Highest FPS - best for real-time applications'
            }
    
    # 3. Most Efficient (Best Parameter Efficiency)
    if 'parameter_efficiency' in df.columns:
        valid_param_eff_data = df.dropna(subset=['parameter_efficiency'])
        if len(valid_param_eff_data) > 0:
            best_param_eff_idx = valid_param_eff_data['parameter_efficiency'].idxmax()
            best_param_eff_model = valid_param_eff_data.loc[best_param_eff_idx]
            
            recommendations['most_efficient'] = {
                'model': best_param_eff_model['model_variant'],
                'parameter_efficiency': best_param_eff_model['parameter_efficiency'],
                'map50': best_param_eff_model.get('map50', 'N/A'),
                'reasoning': 'Best performance per parameter - optimal for resource-constrained environments'
            }
    
    # 4. Best Edge Deployment Model
    edge_suitable = filter_edge_suitable_models(df)
    if len(edge_suitable) > 0 and 'map50' in edge_suitable.columns:
        best_edge_idx = edge_suitable['map50'].idxmax()
        best_edge_model = edge_suitable.loc[best_edge_idx]
        
        recommendations['best_edge'] = {
            'model': best_edge_model['model_variant'],
            'map50': best_edge_model['map50'],
            'fps': best_edge_model.get('fps', 'N/A'),
            'model_size_mb': best_edge_model.get('model_size_mb', 'N/A'),
            'reasoning': 'Best performance while meeting all edge deployment constraints'
        }
    
    # 5. Best Balanced Model (Pareto Optimal)
    if 'fps' in df.columns and 'map50' in df.columns:
        valid_data = df.dropna(subset=['fps', 'map50'])
        if len(valid_data) > 0:
            pareto_points = identify_pareto_optimal(valid_data, 'fps', 'map50')
            
            if len(pareto_points) > 0:
                # Find the Pareto point closest to the "ideal" point (max FPS, max mAP)
                max_fps = pareto_points['fps'].max()
                max_map = pareto_points['map50'].max()
                
                # Calculate distance to ideal point (normalized)
                pareto_points = pareto_points.copy()
                pareto_points['distance_to_ideal'] = np.sqrt(
                    ((pareto_points['fps'] - max_fps) / max_fps) ** 2 + 
                    ((pareto_points['map50'] - max_map) / max_map) ** 2
                )
                
                best_balanced_idx = pareto_points['distance_to_ideal'].idxmin()
                best_balanced_model = pareto_points.loc[best_balanced_idx]
                
                recommendations['best_balanced'] = {
                    'model': best_balanced_model['model_variant'],
                    'map50': best_balanced_model['map50'],
                    'fps': best_balanced_model['fps'],
                    'reasoning': 'Pareto optimal - best balance between performance and speed'
                }
    
    # Print recommendations
    for scenario, rec in recommendations.items():
        print(f"\n🏆 {scenario.replace('_', ' ').title()}:")
        print(f"   Model: {rec['model']}")
        for key, value in rec.items():
            if key not in ['model', 'reasoning']:
                if isinstance(value, float):
                    print(f"   {key}: {value:.4f}")
                else:
                    print(f"   {key}: {value}")
        print(f"   💡 {rec['reasoning']}")
    
    # Create deployment scenarios summary
    print(f"\n📋 DEPLOYMENT SCENARIOS SUMMARY:")
    print(f"   🏭 Manufacturing Line (High Accuracy): {recommendations.get('best_performance', {}).get('model', 'N/A')}")
    print(f"   🚀 Real-time Inspection (High Speed): {recommendations.get('best_speed', {}).get('model', 'N/A')}")
    print(f"   📱 Edge Device (Resource Constrained): {recommendations.get('best_edge', {}).get('model', 'N/A')}")
    print(f"   ⚖️  Balanced Application: {recommendations.get('best_balanced', {}).get('model', 'N/A')}")
    
    return recommendations


# Generate recommendations
recommendations = recommend_models_for_deployment(df)

## 11. Statistical Significance Testing

In [ ]:
def perform_statistical_analysis(df):
    """
    Perform statistical significance testing.
    """
    print("📈 STATISTICAL SIGNIFICANCE ANALYSIS")
    print("=" * 50)
    
    try:
        from scipy.stats import kruskal, mannwhitneyu, ttest_ind
    except ImportError:
        print("⚠️  SciPy not available - skipping statistical tests")
        return
    
    if len(df) == 0:
        print("⚠️  No data available for statistical analysis")
        return
    
    # 1. Model Type Comparison
    if 'model_type' in df.columns and 'map50' in df.columns:
        print("\n🔬 Model Type Performance Comparison:")
        
        valid_data = df.dropna(subset=['model_type', 'map50'])
        model_groups = []
        model_names = []
        
        for model_type in valid_data['model_type'].unique():
            group_data = valid_data[valid_data['model_type'] == model_type]['map50']
            if len(group_data) > 0:
                model_groups.append(group_data)
                model_names.append(model_type)
        
        if len(model_groups) >= 2:
            if len(model_groups) > 2:
                stat, p_value = kruskal(*model_groups)
                test_name = "Kruskal-Wallis"
            else:
                stat, p_value = mannwhitneyu(model_groups[0], model_groups[1], alternative='two-sided')
                test_name = "Mann-Whitney U"
            
            print(f"   Test: {test_name}")
            print(f"   Groups: {model_names}")
            print(f"   Statistic: {stat:.4f}")
            print(f"   P-value: {p_value:.4f}")
            print(f"   Significant (α=0.05): {'Yes' if p_value < 0.05 else 'No'}")
            
            if p_value < 0.05:
                print(f"   ✅ Significant difference detected between model types")
            else:
                print(f"   ➡️  No significant difference between model types")
    
    # 2. Attention Mechanism Comparison
    if 'attention_mechanism' in df.columns and 'map50' in df.columns:
        print("\n🎯 Attention Mechanism Performance Comparison:")
        
        valid_data = df.dropna(subset=['attention_mechanism', 'map50'])
        
        # Compare attention mechanisms vs baseline (none)
        baseline_data = valid_data[valid_data['attention_mechanism'] == 'none']['map50']
        attention_data = valid_data[valid_data['attention_mechanism'] != 'none']['map50']
        
        if len(baseline_data) > 0 and len(attention_data) > 0:
            stat, p_value = mannwhitneyu(baseline_data, attention_data, alternative='two-sided')
            
            print(f"   Test: Mann-Whitney U (Baseline vs Attention)")
            print(f"   Baseline samples: {len(baseline_data)} (mean: {baseline_data.mean():.4f})")
            print(f"   Attention samples: {len(attention_data)} (mean: {attention_data.mean():.4f})")
            print(f"   Statistic: {stat:.4f}")
            print(f"   P-value: {p_value:.4f}")
            print(f"   Significant (α=0.05): {'Yes' if p_value < 0.05 else 'No'}")
            
            improvement = attention_data.mean() - baseline_data.mean()
            if p_value < 0.05:
                print(f"   ✅ Attention mechanisms {'improve' if improvement > 0 else 'reduce'} performance by {abs(improvement):.4f} mAP@0.5")
            else:
                print(f"   ➡️  No significant difference from attention mechanisms")
    
    # 3. Resolution Study Analysis
    if 'image_size' in df.columns and 'map50' in df.columns:
        print("\n📏 Resolution Impact Analysis:")
        
        valid_data = df.dropna(subset=['image_size', 'map50'])
        
        # Compare standard resolution (640) vs high resolution
        standard_res_data = valid_data[valid_data['image_size'] == 640]['map50']
        high_res_data = valid_data[valid_data['image_size'] > 640]['map50']
        
        if len(standard_res_data) > 0 and len(high_res_data) > 0:
            stat, p_value = mannwhitneyu(standard_res_data, high_res_data, alternative='two-sided')
            
            print(f"   Test: Mann-Whitney U (640px vs High Resolution)")
            print(f"   Standard res samples: {len(standard_res_data)} (mean: {standard_res_data.mean():.4f})")
            print(f"   High res samples: {len(high_res_data)} (mean: {high_res_data.mean():.4f})")
            print(f"   Statistic: {stat:.4f}")
            print(f"   P-value: {p_value:.4f}")
            print(f"   Significant (α=0.05): {'Yes' if p_value < 0.05 else 'No'}")
            
            improvement = high_res_data.mean() - standard_res_data.mean()
            if p_value < 0.05:
                print(f"   ✅ High resolution {'improves' if improvement > 0 else 'reduces'} performance by {abs(improvement):.4f} mAP@0.5")
            else:
                print(f"   ➡️  No significant difference from higher resolution")
    
    print(f"\n📊 Statistical analysis complete.")


# Perform statistical analysis
perform_statistical_analysis(df)

## 12. Export Results and Summary

In [ ]:
def export_analysis_results(df, recommendations):
    """
    Export comprehensive analysis results.
    """
    print("💾 Exporting analysis results...")
    
    if len(df) == 0:
        print("⚠️  No data to export")
        return
    
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Export complete results to CSV
    csv_path = OUTPUT_DIR / f"complete_results_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    print(f"   📄 Complete results: {csv_path}")
    
    # 2. Export to Excel with multiple sheets
    try:
        excel_path = OUTPUT_DIR / f"pcb_defect_analysis_{timestamp}.xlsx"
        
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # Main results
            df.to_excel(writer, sheet_name='Complete_Results', index=False)
            
            # Summary statistics
            if 'map50' in df.columns:
                summary_stats = df.groupby('model_variant').agg({
                    'map50': ['count', 'mean', 'std', 'min', 'max'],
                    'map50_95': ['mean', 'std'],
                    'training_time_hours': ['mean', 'std'],
                    'total_parameters': 'first'
                }).round(4)
                
                summary_stats.to_excel(writer, sheet_name='Summary_Statistics')
            
            # Model recommendations
            if recommendations:
                rec_df = pd.DataFrame.from_dict(recommendations, orient='index')
                rec_df.to_excel(writer, sheet_name='Recommendations')
            
            # Pareto optimal models
            if 'fps' in df.columns and 'map50' in df.columns:
                valid_data = df.dropna(subset=['fps', 'map50'])
                if len(valid_data) > 0:
                    pareto_points = identify_pareto_optimal(valid_data, 'fps', 'map50')
                    if len(pareto_points) > 0:
                        pareto_points.to_excel(writer, sheet_name='Pareto_Optimal', index=False)
            
            # Edge suitable models
            edge_suitable = filter_edge_suitable_models(df)
            if len(edge_suitable) > 0:
                edge_suitable.to_excel(writer, sheet_name='Edge_Suitable', index=False)
        
        print(f"   📊 Excel analysis: {excel_path}")
        
    except ImportError:
        print("   ⚠️  openpyxl not available - skipping Excel export")
    
    # 3. Generate comprehensive text report
    report_path = OUTPUT_DIR / f"analysis_report_{timestamp}.txt"
    
    with open(report_path, 'w') as f:
        f.write("PCB DEFECT DETECTION: SYSTEMATIC STUDY ANALYSIS REPORT\n")
        f.write("=" * 80 + "\n")
        f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"WandB Project: {WANDB_PROJECT}\n\n")
        
        # Executive Summary
        f.write("EXECUTIVE SUMMARY\n")
        f.write("-" * 50 + "\n")
        f.write(f"Total Experiments: {len(df)}\n")
        
        if 'map50' in df.columns:
            f.write(f"Average mAP@0.5: {df['map50'].mean():.4f} ± {df['map50'].std():.4f}\n")
            f.write(f"Best Performance: {df['map50'].max():.4f} ({df.loc[df['map50'].idxmax(), 'model_variant']})\n")
        
        if 'training_time_hours' in df.columns:
            f.write(f"Total Training Time: {df['training_time_hours'].sum():.1f} hours\n")
        
        # Model Recommendations
        if recommendations:
            f.write("\nMODEL RECOMMENDATIONS\n")
            f.write("-" * 50 + "\n")
            for scenario, rec in recommendations.items():
                f.write(f"{scenario.replace('_', ' ').title()}: {rec['model']}\n")
                f.write(f"  {rec['reasoning']}\n\n")
        
        # Deployment Guidelines
        f.write("DEPLOYMENT GUIDELINES\n")
        f.write("-" * 50 + "\n")
        f.write("• Manufacturing Line (High Accuracy): Use best performing model\n")
        f.write("• Real-time Inspection (High Speed): Prioritize FPS over accuracy\n")
        f.write("• Edge Devices (Resource Constrained): Consider model size and inference time\n")
        f.write("• Balanced Applications: Use Pareto optimal models\n\n")
        
        f.write("Analysis generated by PCB Defect Detection Analysis Notebook\n")
    
    print(f"   📝 Text report: {report_path}")
    
    print(f"\n✅ All analysis results exported to: {OUTPUT_DIR}")


# Export results
if 'recommendations' in locals():
    export_analysis_results(df, recommendations)
else:
    export_analysis_results(df, {})

## 13. Final Summary and Conclusions

In [ ]:
def generate_final_summary(df):
    """
    Generate final summary and conclusions.
    """
    print("🎉 FINAL ANALYSIS SUMMARY")
    print("=" * 60)
    
    if len(df) == 0:
        print("⚠️  No data available for final summary")
        return
    
    print(f"\n📊 **Experiment Overview:**")
    print(f"   • Total completed experiments: {len(df)}")
    print(f"   • Model variants tested: {df['model_variant'].nunique()}")
    print(f"   • Attention mechanisms: {df['attention_mechanism'].nunique()}")
    
    if 'training_time_hours' in df.columns:
        print(f"   • Total training investment: {df['training_time_hours'].sum():.1f} hours")
    
    # Performance insights
    if 'map50' in df.columns:
        best_model_idx = df['map50'].idxmax()
        best_model = df.loc[best_model_idx]
        
        print(f"\n🏆 **Key Performance Insights:**")
        print(f"   • Best overall performance: {best_model['model_variant']} ({best_model['map50']:.4f} mAP@0.5)")
        print(f"   • Performance range: [{df['map50'].min():.4f}, {df['map50'].max():.4f}] mAP@0.5")
        print(f"   • Average performance: {df['map50'].mean():.4f} ± {df['map50'].std():.4f}")
    
    # Efficiency insights
    edge_suitable = filter_edge_suitable_models(df)
    print(f"\n⚡ **Efficiency Insights:**")
    print(f"   • Models suitable for edge deployment: {len(edge_suitable)}/{len(df)}")
    
    if 'fps' in df.columns:
        valid_fps = df.dropna(subset=['fps'])
        if len(valid_fps) > 0:
            fastest_model_idx = valid_fps['fps'].idxmax()
            fastest_model = valid_fps.loc[fastest_model_idx]
            print(f"   • Fastest inference: {fastest_model['model_variant']} ({fastest_model['fps']:.1f} FPS)")
    
    if 'parameter_efficiency' in df.columns:
        valid_param_eff = df.dropna(subset=['parameter_efficiency'])
        if len(valid_param_eff) > 0:
            most_efficient_idx = valid_param_eff['parameter_efficiency'].idxmax()
            most_efficient = valid_param_eff.loc[most_efficient_idx]
            print(f"   • Most parameter efficient: {most_efficient['model_variant']} ({most_efficient['parameter_efficiency']:.4f} mAP/M params)")
    
    # Research insights
    print(f"\n🔬 **Research Insights:**")
    
    # Attention mechanism effectiveness
    if 'attention_mechanism' in df.columns and 'map50' in df.columns:
        baseline_performance = df[df['attention_mechanism'] == 'none']['map50'].mean()
        attention_performance = df[df['attention_mechanism'] != 'none']['map50'].mean()
        
        if not pd.isna(baseline_performance) and not pd.isna(attention_performance):
            improvement = attention_performance - baseline_performance
            print(f"   • Attention mechanisms {'improve' if improvement > 0 else 'reduce'} performance by {abs(improvement):.4f} mAP@0.5 on average")
    
    # Resolution impact
    if 'image_size' in df.columns and 'map50' in df.columns:
        standard_res_perf = df[df['image_size'] == 640]['map50'].mean()
        high_res_perf = df[df['image_size'] > 640]['map50'].mean()
        
        if not pd.isna(standard_res_perf) and not pd.isna(high_res_perf):
            improvement = high_res_perf - standard_res_perf
            print(f"   • High resolution {'improves' if improvement > 0 else 'reduces'} performance by {abs(improvement):.4f} mAP@0.5 on average")
    
    # Model architecture comparison
    if 'model_type' in df.columns and 'map50' in df.columns:
        model_performance = df.groupby('model_type')['map50'].mean().sort_values(ascending=False)
        if len(model_performance) > 1:
            print(f"   • Best architecture: {model_performance.index[0]} ({model_performance.iloc[0]:.4f} mAP@0.5)")
    
    # Final recommendations
    print(f"\n🎯 **Final Recommendations:**")
    
    if 'map50' in df.columns:
        # Production deployment
        if df['map50'].max() > 0.8:
            print(f"   ✅ Ready for production deployment (mAP@0.5 > 0.8 achieved)")
        elif df['map50'].max() > 0.6:
            print(f"   👍 Good performance achieved, suitable for most applications")
        else:
            print(f"   ⚠️  Consider additional optimization strategies")
    
    print(f"   • Implement model versioning and A/B testing in production")
    print(f"   • Set up continuous monitoring of model performance")
    print(f"   • Consider ensemble methods for critical applications")
    print(f"   • Establish retraining pipelines for model updates")
    
    print(f"\n📁 **Generated Outputs:**")
    print(f"   • Comprehensive comparison table (CSV/Excel)")
    print(f"   • Performance visualization plots")
    print(f"   • Pareto analysis for trade-off optimization")
    print(f"   • Interactive plots (if Plotly available)")
    print(f"   • Statistical significance analysis")
    print(f"   • Model recommendations for different scenarios")
    print(f"   • Edge deployment suitability analysis")
    
    print(f"\n🚀 **Analysis Complete!**")
    print(f"All results saved to: {OUTPUT_DIR}")


# Generate final summary
generate_final_summary(df)

---

## 📋 Analysis Checklist

This notebook has completed the following analysis tasks:

### ✅ **Data Acquisition & Processing**
- [x] Fetch all experimental runs from WandB API
- [x] Extract comprehensive metrics (mAP@0.5, mAP@0.5-0.95, precision, recall, F1)
- [x] Parse model configurations (type, attention mechanism, loss function, image size)
- [x] Clean and preprocess data for analysis

### ✅ **Performance Analysis** 
- [x] Generate summary statistics for all metrics
- [x] Compare model types (YOLOv8n, YOLOv8s, YOLOv10s)
- [x] Evaluate attention mechanism effectiveness
- [x] Analyze resolution impact on performance
- [x] Calculate training and parameter efficiency

### ✅ **Trade-off Analysis**
- [x] **Pareto analysis for mAP vs FPS optimization**
- [x] **Performance vs model size trade-offs**
- [x] Training time vs accuracy analysis
- [x] Parameter efficiency evaluation
- [x] 3D visualization of performance-speed-size relationships

### ✅ **Edge Deployment Analysis**
- [x] Define edge deployment constraints
- [x] Filter models suitable for edge deployment
- [x] Identify Pareto optimal solutions
- [x] **Generate deployment recommendations**

### ✅ **Statistical Validation**
- [x] Significance testing for model comparisons
- [x] Attention mechanism effectiveness validation
- [x] Resolution impact statistical analysis

### ✅ **Visualization & Reporting**
- [x] Performance comparison plots
- [x] **Pareto frontier visualizations**
- [x] Correlation analysis heatmaps
- [x] Interactive 3D plots (if Plotly available)
- [x] Edge deployment suitability charts

### ✅ **Export & Documentation**
- [x] **Comprehensive comparison table (CSV/Excel)**
- [x] Model recommendations for different scenarios
- [x] Statistical analysis results
- [x] Deployment guidelines
- [x] Complete analysis report

---

## 🎯 Key Deliverables

1. **📊 Comprehensive DataFrame** with all experimental results organized for easy comparison
2. **📈 Pareto Plots** showing optimal trade-offs between mAP@0.5 and FPS for edge deployment
3. **🎯 Model Recommendations** for different deployment scenarios (accuracy-critical, real-time, edge, balanced)
4. **📋 Deployment Guidelines** with specific model suggestions for each use case
5. **📊 Interactive Visualizations** for exploring model performance trade-offs
6. **📄 Exportable Results** in multiple formats (CSV, Excel, HTML) for further analysis

This analysis provides everything needed to select the optimal YOLOv8 configuration for your specific PCB defect detection deployment requirements!